## Import Libraries


In [ ]:
!pip install opendatasets
!pip install pandas

you can use my kaggle credentials:

nomielagarde

d8e0f8ad13e663040a0d037b2208f1b6

In [ ]:
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/nomielagarde/huron-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nomielagarde
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/nomielagarde/huron-dataset


100%|██████████| 3.35G/3.35G [02:58<00:00, 20.1MB/s]


In [ ]:
#installing smp libraru
!pip install segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.4 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=64ad328e47fefd56dd1dc5a6e5cc1ed45212c12c2324eda7327042e1834c3ec1
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=ce39da0c3d19f5430e3c79e9f0ceb7862037c8dc8a6559bdf38a5f303542e9be
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorc

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt

## Class Definitions

### Dataset

In [ ]:
class HuronDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.transform = transform
        self.mask_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ])
        self.images = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.image_dir / self.images[idx]
        mask_path = self.mask_dir / self.images[idx]

        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        if self.transform:
            image = self.transform(image)
        mask = self.mask_transform(mask)

        # Ensure mask is binary
        mask = (mask > 0.5).float()

        return image, mask



### Eval metrics

In [ ]:
def calculate_iou(outputs, targets, threshold=0.5):
    outputs = (outputs > threshold).float()
    targets = targets.float()

    intersection = (outputs * targets).sum(dim=(1, 2))
    union = outputs.sum(dim=(1, 2)) + targets.sum(dim=(1, 2)) - intersection

    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou.mean()

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_iou = 0
    num_batches = len(dataloader)

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)
            iou = calculate_iou(outputs, masks)

            total_loss += loss.item()
            total_iou += iou.item()

    return total_loss / num_batches, total_iou / num_batches

class CombinedLoss(nn.Module):
    def __init__(self, loss1, loss2, weight1=1.0, weight2=1.0):
        super(CombinedLoss, self).__init__()
        self.loss1 = loss1
        self.loss2 = loss2
        self.weight1 = weight1
        self.weight2 = weight2

    def forward(self, y_pred, y_true):
        return self.weight1 * self.loss1(y_pred, y_true) + self.weight2 * self.loss2(y_pred, y_true)


### Visualization

In [ ]:
def visualize_results(model, test_loader, device, num_images=3):
    model.eval()

    # Get some test images
    test_images, test_masks = next(iter(test_loader))

    with torch.no_grad():
        test_images = test_images.to(device)
        predictions = model(test_images)
        predictions = (predictions > 0.5).float()

    # Convert tensors to numpy for visualization
    test_images = test_images.cpu()
    test_masks = test_masks.cpu()
    predictions = predictions.cpu()

    # Plot results
    fig, axes = plt.subplots(num_images, 3, figsize=(15, 5*num_images))

    for idx in range(num_images):
        # Original image
        img = test_images[idx].permute(1, 2, 0)
        img = img * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
        img = img.numpy()
        img = np.clip(img, 0, 1)

        # Ground truth mask
        mask = test_masks[idx].squeeze().numpy()

        # Predicted mask
        pred = predictions[idx].squeeze().numpy()

        # Plot
        axes[idx, 0].imshow(img)
        axes[idx, 0].set_title('Original Image')
        axes[idx, 0].axis('off')

        axes[idx, 1].imshow(mask, cmap='gray')
        axes[idx, 1].set_title('Ground Truth Mask')
        axes[idx, 1].axis('off')

        axes[idx, 2].imshow(pred, cmap='gray')
        axes[idx, 2].set_title('Predicted Mask')
        axes[idx, 2].axis('off')

    plt.tight_layout()
    plt.show()

## Training function

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    best_val_iou = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_iou = 0.0

        with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}') as pbar:
            for images, masks in pbar:
                images = images.to(device)
                masks = masks.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, masks)
                iou = calculate_iou(outputs, masks)

                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                running_iou += iou.item()

                pbar.set_postfix({
                    'loss': running_loss / (pbar.n + 1),
                    'IoU': running_iou / (pbar.n + 1)
                })

        # Validate
        model.eval()
        val_iou = 0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)
                outputs = model(images)
                val_iou += calculate_iou(outputs, masks).item()

        val_iou /= len(val_loader)
        print(f'Validation IoU: {val_iou:.4f}')

        if val_iou > best_val_iou:
            best_val_iou = val_iou
            torch.save(model.state_dict(), 'best_model.pth')

## AdamW

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Dataset paths
    image_dir = '/content/huron-dataset/Sliced_Images'
    mask_dir = '/content/huron-dataset/Sliced_masks'

    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)

    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size

    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

    # Model configuration
    encoder_name = "efficientnet-b4"

    # Initialize U-Net model
    model = smp.UnetPlusPlus(
        encoder_name=encoder_name,        # try different encoders
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights
        in_channels=3,                  # model input channels
        classes=1                      # model output channels
    ).to(device)


    # Initialize loss and optimizer
    criterion = CombinedLoss(
        smp.losses.DiceLoss(mode='binary'),
        nn.BCEWithLogitsLoss())
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

    print(f"Model: U-Net++ with {encoder_name} encoder and AdamW Optimizer")

    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth
100%|██████████| 74.4M/74.4M [00:02<00:00, 36.0MB/s]


Model: U-Net++ with efficientnet-b4 encoder and AdamW Optimizer


Epoch 1/5: 100%|██████████| 1518/1518 [02:46<00:00,  9.10it/s, loss=0.196, IoU=0.897]


Validation IoU: 0.9092


...

Validation IoU: 0.9155


Epoch 5/5: 100%|██████████| 1518/1518 [02:43<00:00,  9.28it/s, loss=0.147, IoU=0.914]


Validation IoU: 0.9174
Evaluating on test set...


<ipython-input-5-54a2804410d2>:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.1493, Test IoU: 0.9163


## RMSProp

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Dataset paths
    image_dir = '/content/huron-dataset/Sliced_Images'
    mask_dir = '/content/huron-dataset/Sliced_masks'

    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)

    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size

    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

    # Model configuration
    encoder_name = "efficientnet-b4"

    # Initialize U-Net model
    model = smp.UnetPlusPlus(
        encoder_name=encoder_name,        # try different encoders
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights
        in_channels=3,                  # model input channels
        classes=1                      # model output channels
    ).to(device)


    # Initialize loss and optimizer
    criterion = CombinedLoss(
        smp.losses.DiceLoss(mode='binary'),
        nn.BCEWithLogitsLoss())
    optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99)

    print(f"Model: U-Net++ with {encoder_name} encoder and AdamW Optimizer")

    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603
Model: U-Net++ with efficientnet-b4 encoder and AdamW Optimizer


Epoch 1/5: 100%|██████████| 1518/1518 [02:41<00:00,  9.41it/s, loss=0.192, IoU=0.896]


Validation IoU: 0.9110


Epoch 2/5: 100%|██████████| 1518/1518 [02:41<00:00,  9.38it/s, loss=0.16, IoU=0.909]


Validation IoU: 0.9115


...

Validation IoU: 0.9065


Epoch 5/5: 100%|██████████| 1518/1518 [02:42<00:00,  9.36it/s, loss=0.146, IoU=0.915]


Validation IoU: 0.9169
Evaluating on test set...


<ipython-input-6-8c98671621a0>:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.1487, Test IoU: 0.9159


## SGD

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Dataset paths
    image_dir = '/content/huron-dataset/Sliced_Images'
    mask_dir = '/content/huron-dataset/Sliced_masks'

    # Transform for input images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Create full dataset
    full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)

    # Calculate splits
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size

    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

    # Model configuration
    encoder_name = "efficientnet-b4"

    # Initialize U-Net model
    model = smp.UnetPlusPlus(
        encoder_name=encoder_name,        # try different encoders
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights
        in_channels=3,                  # model input channels
        classes=1                      # model output channels
    ).to(device)


    # Initialize loss and optimizer
    criterion = CombinedLoss(
        smp.losses.DiceLoss(mode='binary'),
        nn.BCEWithLogitsLoss())
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    print(f"Model: U-Net++ with {encoder_name} encoder and AdamW Optimizer")

    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, device)

    # Evaluate on test set
    print("Evaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_iou = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test IoU: {test_iou:.4f}')

if __name__ == '__main__':
    main()

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603
Model: U-Net++ with efficientnet-b4 encoder and AdamW Optimizer


Epoch 1/5: 100%|██████████| 1518/1518 [02:41<00:00,  9.40it/s, loss=0.225, IoU=0.883]


Validation IoU: 0.9044


Epoch 2/5: 100%|██████████| 1518/1518 [02:40<00:00,  9.46it/s, loss=0.174, IoU=0.903]


Validation IoU: 0.9084


...

Validation IoU: 0.9050


Epoch 5/5: 100%|██████████| 1518/1518 [02:40<00:00,  9.45it/s, loss=0.154, IoU=0.91]


Validation IoU: 0.9076
Evaluating on test set...


<ipython-input-7-1cff252de3c9>:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.1591, Test IoU: 0.9076
